In [6]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE 
from qiskit.algorithms.optimizers import SLSQP 
from qiskit.circuit.library import TwoLocal 
from qiskit.quantum_info import SparsePauliOp 
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
#pip install qiskit[nature]
import qiskit_nature
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
qiskit_nature.settings.use_pauli_sum_op = False
#install psi4 library   
from qiskit_nature.second_q.drivers import Psi4Driver
#pip install qiskit-ibm-runtime
from qiskit_ibm_runtime import qiskit_runtime_service, Estimator

In [7]:
#hamiltonian = SparsePauliOp.from_list([('II', -1), ('IZ', 0.3), ('XI', -0.3), ('ZY', -0.01), ('YX', 0.1)]) 

def qubit_operator(dist):
    # Define Molecule
    mol = MoleculeInfo(
        symbols=["H", "H"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  #2*spin + 1
        charge=0,
    )

    prob_unmod = Psi4Driver.from_molecule(mol).run()

    # reducing electron structure by symmetry and Born–Oppenheimer approximation
    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(prob_unmod)

    particle_count = problem.num_particles
    spatial_orbitals_count = problem.num_spatial_orbitals

    mapper = ParityMapper(num_particles=particle_count)
    hamiltonian = mapper.map(problem.second_q_ops()[0])
    return hamiltonian, particle_count, spatial_orbitals_count, problem, mapper

In [8]:
def exact_solver(hamiltonian, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(hamiltonian)
    result = problem.interpret(sol)
    return result

In [9]:
service = QiskitRuntimeService(channel="ibm_quantum", token="68e741eb4ff82f98673409939261d735201636fa51b3654fc5a8dd08359651981481178017c543bc4e92dd12388951e22c2ae95d95b3012d19c185efe132d815")
backend =  service.get_backend("ibmq_naomi")
estimator = Estimator(session=backend) 
optimizer = SLSQP() 
#ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz') 
#vqe = VQE(estimator, ansatz, optimizer)

In [ ]:
exact_energies = []
vqe_energies = []
circ_ls = []

for i in np.arange(0.3, 2.5, 0.1):
    get_op = qubit_operator(i)

    init_state = HartreeFock(get_op[2], get_op[1], get_op[4])
    ansatz = UCCSD(
        get_op[2], get_op[1], get_op[4], initial_state=init_state
    )
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)
    result = vqe.compute_minimum_eigenvalue(operator=get_op[0]) 
    print(result)
    np_result = exact_solver(get_op[0], get_op[3]).total_energies[0].real
    exact_energies.append(np_result)
    circ_ls.append(result.optimal_circuit)
    vqe_calc = vqe.compute_minimum_eigenvalue(operator=get_op[0])
    vqe_result = get_op[3].interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print('distance:',i,'vqe energy:', vqe_result, 'actual:', np_result)

<img src = images/graph.png width = "600" height = "400" >
<img src = images/eqn.png  >

In [ ]:
display(circ_ls[1].decompose().draw(output='mpl'))
final_array = np.array(list(result.optimal_parameters.items()))

plt.figure(1, figsize=( 4, 0.218*np.shape(final_array)[0]))
tb = plt.table(cellText=final_array, loc=(0,0), cellLoc='center')

ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])

plt.show()

The output above prints Ansatz (our simple twolocal visualized here) fit with parameter. optimal_parameters giving eigenvalue are also given below the diagram